In [5]:
import json

# Load the JSON data
with open('./kb2.json', 'r') as file:
    kb = json.load(file)

# Function to query rules based on a condition
def query_rules_by_condition(condition):
    results = []
    for rule, details in kb['rules'].items():
        if condition in details['conditions']:
            results.append({
                "rule": rule,
                "action": details['action']
            })
    return results

# Function to get the action for a specific rule
def get_action_by_rule(rule_name):
    rule = kb['rules'].get(rule_name)
    if rule:
        return rule['action']
    else:
        return "Rule not found."

def get_compensation_details(rule_name):
    rule = kb['rules'].get(rule_name)
    if not rule:
        return {"error": "Rule not found."}

    compensation = rule.get("compensation")
    if not compensation:
        return {"error": "Compensation details not defined for this rule."}

    return compensation

# Example Usage
condition_to_check = "no_heat_water_or_electricity"
print("Rules matching condition:")
print(query_rules_by_condition(condition_to_check))

rule_to_check = "habitability_violation"
print(f"Action for rule '{rule_to_check}':")
print(get_action_by_rule(rule_to_check))

print(get_compensation_details(rule_to_check))


Rules matching condition:
[{'rule': 'failure_to_provide_essential_services', 'action': 'tenant_entitled_to_substitute_housing_or_rent_deduction'}]
Action for rule 'habitability_violation':
tenant_entitled_to_rent_reduction_or_abatement
{'type': 'percentage', 'value': '25-100', 'calculation_details': 'Percentage of rent abated based on severity and duration of habitability issue.'}


In [6]:
import json
from typing import Literal
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.prompts import PromptTemplate
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [8]:
class RuleSelection(BaseModel):
    selected_rule: Literal[
        "habitability_violation",
        "security_deposit_issue",
        "failure_to_provide_essential_services",
        "retaliatory_eviction",
        "illegal_lockout",
        "failure_to_address_health_hazard",
        "unauthorized_entry_by_landlord",
        "notice_of_foreclosure"
    ]

# Prompt template for LLM
prompt_template = """
You are an expert in tenant-landlord laws. Based on the details of a case, select the most applicable rule from the following knowledge base:

1. habitability_violation:
   - Conditions: Unresolved maintenance issue, health hazard, tenant notified landlord, no repair in reasonable time.
   - Action: Tenant entitled to rent reduction or abatement.

2. security_deposit_issue:
   - Conditions: Security deposit not returned, tenant vacated unit, no damages or unpaid rent.
   - Action: Tenant entitled to double the security deposit.

3. failure_to_provide_essential_services:
   - Conditions: No heat, water, or electricity; tenant provided written notice; landlord noncompliant for 24 hours.
   - Action: Tenant entitled to substitute housing or rent deduction.

4. retaliatory_eviction:
   - Conditions: Tenant reported violation, landlord issued eviction notice, within six months of the report.
   - Action: Tenant entitled to remain and landlord fined.

5. illegal_lockout:
   - Conditions: Tenant locked out without court order, lock changed or entry blocked.
   - Action: Tenant entitled to reentry and damages.

6. failure_to_address_health_hazard:
   - Conditions: Mold or pest issue, tenant has medical report, inspection confirms issue.
   - Action: Tenant entitled to injunctive relief and damages.

7. unauthorized_entry_by_landlord:
   - Conditions: Landlord entered without notice, no emergency or consent.
   - Action: Tenant entitled to damages for privacy violation.

8. notice_of_foreclosure:
   - Conditions: Landlord facing foreclosure, tenant not informed in writing.
   - Action: Tenant entitled to terminate the lease.

Given the following case details:
{case_details}

Respond with only the rule name.
"""

# llm = OllamaFunctions(model="llama3-groq-tool-use", temperature=0)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_key)
structured_llm = llm.with_structured_output(RuleSelection)

rules_prompt = PromptTemplate(input_variables=['case_details'], template=prompt_template)

rules_chain = rules_prompt | structured_llm

case_details_input = """
On January 5th, 2024, the tenant, Jane Doe, reported a lack of heating in her apartment located at 123 Main Street, Cityville, to her landlord, John Smith, via email. Despite providing written notice and follow-ups, no repairs have been made to restore heating. The issue has persisted for five days, leaving the tenant without adequate heating during a severe winter cold spell. Additionally, the tenant has no access to alternative heating sources due to electrical system limitations, creating unsafe living conditions.
"""

res = rules_chain.invoke({"case_details": case_details_input})

In [9]:
res.selected_rule

'failure_to_provide_essential_services'

In [10]:
get_action_by_rule(res.selected_rule)

'tenant_entitled_to_substitute_housing_or_rent_deduction'

In [11]:
get_compensation_details(res.selected_rule)

{'type': 'fixed_or_percentage',
 'value': 'substitute_housing_cost or 50% rent reduction',
 'calculation_details': 'Substitute housing costs reimbursed or rent reduced by 50% depending on the severity and duration of the issue.'}

### Decision readiness

In [12]:
class DecisionReadiness(BaseModel):
    decision_ready: Literal["yes", "no"]

In [18]:
prompt_template = """
You are an expert in tenant-landlord laws. Your task is to determine if there is enough information in the given conversation between the user and the LLM to reasonably assess which rule applies. The rules are based on the following conditions:

1. Habitability Violation:
   - Conditions: Unresolved maintenance issue, health hazard, tenant notified landlord, no repair in reasonable time.

2. Security Deposit Issue:
   - Conditions: Security deposit not returned, tenant vacated unit, no damages or unpaid rent.

3. Failure to Provide Essential Services:
   - Conditions: No heat, water, or electricity; tenant provided written notice; landlord noncompliant for 24 hours.

4. Retaliatory Eviction:
   - Conditions: Tenant reported violation, landlord issued eviction notice, within six months of the report.

5. Illegal Lockout:
   - Conditions: Tenant locked out without court order, lock changed or entry blocked.

6. Failure to Address Health Hazard:
   - Conditions: Mold or pest issue, tenant has medical report, inspection confirms issue.

7. Unauthorized Entry by Landlord:
   - Conditions: Landlord entered without notice, no emergency or consent.

8. Notice of Foreclosure:
   - Conditions: Landlord facing foreclosure, tenant not informed in writing.

To decide if there is enough information:
- All conditions for at least one rule should be met or clearly implied.
- If most required conditions for one rule are met or strongly implied, respond with "yes."
- If critical conditions are missing or highly ambiguous, respond with "no."

Given the following conversation:
{conversation}

Respond with only "yes" or "no".
"""

# llm = OllamaFunctions(model="llama3-groq-tool-use", temperature=0)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_key)
structured_llm = llm.with_structured_output(DecisionReadiness)

decision_prompt = PromptTemplate(input_variables=['conversation'], template=prompt_template)

decision_chain = decision_prompt | structured_llm

In [19]:
# Sufficient information
conversation = """
User: My landlord has been refusing to return my security deposit even though I moved out two months ago.
LLM: Did you leave the unit in good condition, with no damages or unpaid rent?
User: Yes, I cleaned the apartment thoroughly, and I don’t owe any rent.
LLM: Did you ask your landlord about the deposit, and what was their response?
User: Yes, I asked multiple times, but they keep ignoring my messages.
"""

res = decision_chain.invoke({'conversation': conversation})
res.decision_ready

'yes'

In [20]:
# Insufficient Information
conversation = """
User: My apartment has a maintenance issue that hasn’t been resolved for a while.
LLM: Could you describe the issue in more detail? Is it affecting your health or safety?
User: It’s just a plumbing issue, but it’s inconvenient.
LLM: Have you notified your landlord about this issue? If so, how long has it been since you informed them?
User: I haven’t told them yet; I was hoping they’d notice during their visits.
"""

res = decision_chain.invoke({'conversation': conversation})
res.decision_ready

'no'